# Results Analysis - Use Old Summary Statistics and Similarity Measure 

In [ ]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.chdir('../')

In [ ]:
from scipy.signal import argrelmin, argrelmax

In [ ]:
from dap import DAPcython
from dap.utils import obs_params, load_current
from tqdm import tqdm
from scipy.spatial import distance

### Set Parameters

In [ ]:
dt = 1e-2
params, labels = obs_params(reduced_model=False)
data_dir = '/home/alteska/Desktop/LFI_DAP/data/rawData/2015_08_26b.dat'

### load the file

In [ ]:
directory = './parameters/'
dir = glob.glob(directory + '*')

In [ ]:
fname_start = dir[0].find('dap_')
fname_stop = dir[0].find('n_')
fname = dir[0][fname_start:fname_stop] + '.csv'

df_param = pd.read_csv(fname)

### calculate DAP

In [ ]:
# load the input data
Ir, vr, tr, t_onr, t_offr, dtr = load_current(data_dir, protocol='rampIV', ramp_amp=3.1)
Is, vs, ts, t_ons, t_offs, dts = load_current(data_dir, protocol='IV', ramp_amp=1)

# define a model
dap = DAPcython(-75, params)

In [ ]:
# run models on original parameters
U_step = dap.simulate(dts, ts, Is)
U_ramp = dap.simulate(dtr, tr, Ir)

### calculate the similarities

In [ ]:
d_step = distance.euclidean(vs, U_step)
d_step

In [ ]:
d_ramp = distance.euclidean(vr, U_ramp)
d_ramp

In [ ]:
d_step+d_ramp

### run for all cells and save into the the DF

In [ ]:
df_paramT = df_param.transpose()
df_paramT.head()

In [ ]:
df_paramT.drop('Unnamed: 0', inplace=True)
df_paramT.head()

In [ ]:
daps = []
U_steps = []
U_ramps = []

for i, j in tqdm(df_paramT.iterrows()):
    # get parameters
    par_temp = j.values

    # define a model
    daps.append(DAPcython(-75, j))

    # run model
    U_steps.append(dap.simulate(dts, ts, Is))
    U_ramps.append(dap.simulate(dtr, tr, Ir))

## Calculating the features for each parameter

In [ ]:
def calc_ramp_sum_stats(v, t, dt, t_on, t_off):
    """Calculate summary statistics of a single run"""
    stats = []
    stats_idx = []
#     v = v.transpose()
    N = v.shape[0]

    # resting potential
    rest_pot = np.mean(v[t<t_on])
    rest_pot_std = np.std(v[int(.9*t_on/dt):int(t_on/dt)])   # TODO: add if needed

    # RMSE
#     n = len(self.v0)
#     rmse = np.linalg.norm(v - self.v0) / np.sqrt(n)

    # more then one AP:
    multiple_AP = np.shape(np.where(v > 0))[1]

    #case without any action potential or more then one AP
    if (np.all(v <= 20)):
        AP_onsets = 999
        AP_amp = 999
        AP_width = 999
        DAP_amp = 999
        DAP_width = 999
        DAP_deflection = 999
        DAP_time = 999
        mAHP = 999
        fAHP = 999

    else:
        threshold = -30
        # hyperpolarization after DAP
        mAHP_idx = np.argmin(v)
        mAHP = v[mAHP_idx]

        # Action potential
        AP_onsets = np.where(v > threshold)[0]
        AP_start = AP_onsets[0]
        AP_end = AP_onsets[-1]
        AP_max_idx = AP_start + np.argmax(v[AP_start:AP_end])
        AP_max = v[AP_max_idx]
        AP_amp = AP_max - rest_pot

        # AP width
        AP_onsets_half_max = np.where(v > (AP_max+rest_pot)/2)[0]
        AP_width = t[AP_onsets_half_max[-1]] - t[AP_onsets_half_max[0]]

        # DAP: fAHP
        v_dap = v[AP_max_idx:]

        fAHP_idx = argrelmin(v[AP_max_idx:])[0][0] + AP_max_idx
        fAHP = v[fAHP_idx]

        # DAP amplitude
        DAP_max_idx = argrelmax(v_dap)[0][1] + AP_max_idx
        DAP_max = v[DAP_max_idx]
        DAP_amp = DAP_max - rest_pot

        DAP_deflection = DAP_amp - (fAHP - rest_pot)
        DAP_time = t[DAP_max_idx] - t[AP_max_idx]    # Time between AP and DAP maximum

        # Width of DAP: between fAHP and halfsize of fAHP after DAP max
        vnorm = v[DAP_max_idx:] - rest_pot

        if np.any((abs(vnorm) < abs(fAHP - rest_pot)/2)):
            half_max = np.where((abs(vnorm) < abs(fAHP - rest_pot)/2))[0]

            DAP_width_idx = DAP_max_idx + half_max[0]
            DAP_width = (DAP_width_idx - fAHP_idx) * dt
        else:
            DAP_width = 999


    sum_stats_vec = np.array([
                    rest_pot,
                    AP_amp,
                    AP_width,
                    fAHP,
                    DAP_amp,
                    DAP_width,
                    DAP_deflection,
                    DAP_time,
                    mAHP,
                    ])


    return sum_stats_vec

In [ ]:
sim = calc_ramp_sum_stats(U_ramps[0], tr, dtr, t_onr, t_offr)
pd.DataFrame(data=sim)

## Features Step Current: Outside of The Function
to pick required parameters

In [ ]:
v, t, dt, t_on, t_off = U_steps[0], ts, dts, t_ons, t_offs

"""Calculate summary statistics"""
stats = []
N = v.shape[0]
N

In [ ]:
plt.plot(v);

In [ ]:
# put everything to -10 that is below -10 or has negative slope
ind = np.where(v < -10)
v[ind] = -10
ind = np.where(np.diff(v) < 0)
v[ind] = -10

In [ ]:
plt.plot(v);

In [ ]:
np.diff(v.transpose())

In [ ]:
v = v.transpose()

In [ ]:
np.diff(v)

In [ ]:
# remaining negative slopes are at spike peaks
ind = np.where(np.diff(v) > 0)
ind = ind[1]

In [ ]:
# choose one spike time within close spike times (window of 0.5 ms)
ind1 = np.array(ind)
ind1

In [ ]:
spike_times = np.array(t)[ind]
spike_times

In [ ]:
spike_times_stim = spike_times[(spike_times > t_on) & (spike_times < t_off)]
spike_times_stim

In [ ]:
ind_stim1 = ind1[(spike_times > t_on) & (spike_times < t_off)]
ind_stim1

In [ ]:
# ind_stim1 = ind_stim1[np.append(1,np.diff(spike_times_stim))>0.5] # ??????????????????????
ind_stim = ind_stim1.astype(int)
ind_stim

In [ ]:
# WHY THIS?
# spike_times_stim = spike_times_stim[np.append(1,np.diff(spike_times_stim))>0.5]
# spike_times_stim

In [ ]:
# firing rate
firing_rate = 1e3*np.absolute(spike_times_stim.shape[0]/(t_off-t_on))

time_1st_spike = spike_times_stim[spike_times_stim>t_on][0]

In [ ]:
# average spike width
if spike_times_stim.shape[0] == 1:
    delta_ind_spik = np.round(t[(t>t_on) & (t<t_off)].shape[0]/2).astype(int)
else:
    ISI = np.diff(spike_times_stim).astype(float)
    delta_ind_spik = np.round(np.min(ISI)/(2*dt)).astype(int)    

In [ ]:
ISI

In [ ]:
delta_ind_spik

In [ ]:
v = v.transpose()

In [ ]:
spike_width1 = np.zeros_like(spike_times_stim)
for i_sp in range(spike_times_stim.shape[0]):
    # voltages post-spike
    x_isi = v[ind_stim[i_sp].astype(int):np.minimum(ind_stim[i_sp].astype(int)+delta_ind_spik,N)]
    t_isi = t[ind_stim[i_sp].astype(int):np.minimum(ind_stim[i_sp].astype(int)+delta_ind_spik,N)]
    
    x_post = x_isi[0:np.maximum(np.argmin(x_isi),2)]
    t_post = t_isi[0:np.maximum(np.argmin(x_isi),2)]

   
    # half-maximum voltage
    x_half_max = 0.5*(x_post[-1]+x_post[0])

    # voltages pre-spike
    x_pre = v[np.maximum(ind_stim[i_sp].astype(int)-delta_ind_spik,0):ind_stim[i_sp].astype(int)]
    t_pre = t[np.maximum(ind_stim[i_sp].astype(int)-delta_ind_spik,0):ind_stim[i_sp].astype(int)]

    spike_width1[i_sp] = t_post[np.argmin(np.absolute(x_post - x_half_max))]-t_pre[np.argmin(np.absolute(x_pre - x_half_max))]

spike_width = np.mean(spike_width1)

In [ ]:
spike_width

In [ ]:
t_begin = t[(t>t_on) & (t<time_1st_spike)]
v_begin = v[(t>t_on) & (t<time_1st_spike)]
t_begin

In [ ]:
np.diff(v_begin.transpose(), n=2)

In [ ]:
# latency from stimulus onset to first spike and mean action potential overshoot
if t_begin.shape[0] == 0:
    AP_latency = 10000
    AP_overshoot_mn = np.max(v[(t > t_on) & (t < t_off)])
elif t_begin.shape[0] == 1 or t_begin.shape[0] == 2:
    AP_latency = np.absolute(t_begin[0])
    AP_overshoot_mn = np.absolute(np.mean(v[ind_stim]))
else:
    AP_latency = np.absolute(t_begin[np.argmax(np.diff(v_begin.transpose(), n=2))])
    AP_overshoot_mn = np.absolute(np.mean(v[ind_stim]))

# resting potential mean and std
rest_pot = np.mean(v[t<t_on])
rest_pot_std = np.std(v[int(.9*t_on/dt):int(t_on/dt)])

# ISI mean and std
if spike_times_stim.shape[0] > 1:
    ISImom = np.array([np.mean(ISI),np.std(ISI)])
else:
    ISImom = np.array([1000.]*2)
ISI1 = np.array([1000.]*2)
ISI1[0:np.maximum(0,spike_times_stim.shape[0]-1)] = ISImom[0:np.maximum(0,spike_times_stim.shape[0]-1)]
